In [3]:
import pandas as pd
import subprocess
import os

# Excel-Datei laden oder neue erstellen
excel_file = "idn.xlsx"
if os.path.exists(excel_file):
    idn_df = pd.read_excel(excel_file, dtype=str)
else:
    idn_df = pd.DataFrame(columns=["IDN"])

# PICA-RS Dump-Datei
datendump_file = r"D:\Pica-RS\DNBtitel.dat"  # Rohstring für Windows-Pfad
filtered_dump = r"D:\Pica-RS\DNBtitel_filtered.dat"  # Zwischen-Dump

# 1. Filterbefehl: neuen Datendump erstellen
filter_cmd = f'pica filter -s  047I.c=04"' "{datendump_file}" -o "{filtered_dump}"

try:
    subprocess.run(filter_cmd, check=True, shell=True)
    print("Gefilterter Dump erstellt:", filtered_dump)

    # 2. Select-Befehl: nur IDNs extrahieren
    select_cmd = f'pica-rs select "{filtered_dump}" 003@.0'
    result = subprocess.run(select_cmd, capture_output=True, text=True, shell=True)
    result.check_returncode()

    idns_new = result.stdout.splitlines()

    # Nur neue IDNs hinzufügen
    idns_to_add = [idn for idn in idns_new if idn not in idn_df["IDN"].values]

    if idns_to_add:
        new_df = pd.DataFrame(idns_to_add, columns=["IDN"])
        idn_df = pd.concat([idn_df, new_df], ignore_index=True)
        idn_df.to_excel(excel_file, index=False)
        print(f"{len(idns_to_add)} neue IDNs hinzugefügt.")
    else:
        print("Keine neuen IDNs gefunden.")

except subprocess.CalledProcessError as e:
    print("Fehler beim Ausführen von PICA-RS:", e)


SyntaxError: invalid syntax (211845823.py, line 17)